# Imports

In [ ]:
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import mixed_precision

# Load Dataset

In [ ]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
batch_size = 64
img_height = 224
img_width = 224

In [ ]:
train_dataset, validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

class_names = train_dataset.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Create model and fit 

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
start_time = time.time()

model = Sequential([
    layers.Input((img_height, img_width, 3)),
    layers.Rescaling(1./255),
    data_augmentation,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']);
model.summary()
epochs = 30
history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=epochs
)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

# Results for custom model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Create MobileNetV1 model

In [ ]:
from tensorflow.keras.models import Model

start_time = time.time()

preprocessing_input = layers.Input(shape=(img_height, img_width, 3))
x = layers.Rescaling(1./255)(preprocessing_input)
x = data_augmentation(x)  # Apply data augmentation
preprocessing = Model(inputs=preprocessing_input, outputs=x)

base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

input_tensor = preprocessing(preprocessing_input)
x = base_model(input_tensor, training=False)  # Apply base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Add pooling layer
x = layers.Dense(512, activation='relu')(x)  # Add dense layer
predictions = layers.Dense(len(class_names), activation='softmax')(x)  # Output layer

model = Model(inputs=preprocessing_input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
model.summary()
# Train the model
epochs = 30
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

# Results for MobileNetV1 model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Create EfficientNetB0

In [ ]:
from tensorflow.keras.models import Model

start_time = time.time()

preprocessing_input = layers.Input(shape=(img_height, img_width, 3))
x = layers.Rescaling(1./255)(preprocessing_input)
x = data_augmentation(x)  # Apply data augmentation
preprocessing = Model(inputs=preprocessing_input, outputs=x)

base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

input_tensor = preprocessing(preprocessing_input)
x = base_model(input_tensor, training=False)  # Apply base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Add pooling layer
x = layers.Dense(512, activation='relu')(x)  # Add dense layer
predictions = layers.Dense(len(class_names), activation='softmax')(x)  # Output layer

model = Model(inputs=preprocessing_input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
model.summary()
# Train the model
epochs = 30
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

# Results for EfficientNetB0

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()